In [2]:
# 2021-09-28
# 09-Animate-Ecliptic-save-Hrs
#
# When into animaton mode, code error massage will not display, include pint()
#
# How to guide ...
# https://holypython.com/how-to-save-matplotlib-animations-the-ultimate-guide/
#
# Enable interactive plot
%matplotlib widget
#
import numpy  as np 
import pandas as pd
import matplotlib.pyplot as plt
#
from IPython.display  import HTML
from shapely.geometry import LineString
from matplotlib import animation
# 
# 定義圖面函數 ------------------------------------------------------
def set_graphic_area(width,height) :
    #    
    cm2inch = 1/2.54    # inch per cm
    #
    # define graphic area
    #
    left_margin  = 2.0  # cm
    right_margin = 1.0  # cm
    #
    figure_width  = width  # cm , from xmin to xmax
    figure_height = height # cm , from ymin to ymax
    #
    top_margin = 1.0    # cm
    bottom_margin = 1.5 # cm
    #
    box_width = left_margin + figure_width + right_margin   # cm
    box_height = top_margin + figure_height + bottom_margin # cm
    #
    top_value    = 1.0 - top_margin / box_height
    bottom_value = bottom_margin / box_height
    left_value   = left_margin / box_width
    right_value  = 1.0 - right_margin / box_width
    #
    return (box_width*cm2inch,box_height*cm2inch,top_value,bottom_value,left_value,right_value,width)
#
# end of def
#
# Define the arc
# center position is cxy = (cx, cy)
# start_angle, end_angle is in degrees
#
def shapely_Arc(cxy, r, start_angle, end_angle) :
    #
    numsegments = 1440
    # The coordinates of the arc
    theta = np.radians(np.linspace(start_angle, end_angle, numsegments))
    x = cxy[0] + r * np.cos(theta)
    y = cxy[1] + r * np.sin(theta)
    Arc = LineString(np.column_stack([x, y]))
    return Arc
    #
# end of def
#
def motif_line(LS, COLOR, ZORDER, LINEWIDTH) :
    #
    xs, ys = LS.xy
    ax.plot(xs, ys, color=COLOR, alpha=0.7, linewidth=LINEWIDTH, solid_capstyle='round', zorder=ZORDER)
    return
#
# find number of multiples of 30° (π/6)
x_ticks_tmp = np.linspace(0,360,12+1) # for 2 hour
x_ticks = []
for item in map(int, x_ticks_tmp) :
    x_ticks.append(item)
# end for
#
def format_func(value, tick_number):
    N = int(np.round(6 * value / np.pi))
    return '$' + str(2*N) + '^h$'
# end def
#
# 設定圖面大小
#  tup7 is type of Tuple, have 7 indexed value
#
tup7 = set_graphic_area(6*np.pi,6)  
# 呼叫 pyplot.figure(), 建立一個圖表物件, 並成為目前圖表物件
fig = plt.figure(figsize=(tup7[0], tup7[1])) 
# 圖表的繪圖區域被分為1個子圖, 1 row, 1 column, plot to index 1
ax = fig.add_subplot(1,1,1)   
fig.subplots_adjust(
                top    = tup7[2] ,
                bottom = tup7[3] ,
                left   = tup7[4] ,
                right  = tup7[5] ,
                )
#
plt.xlim(-np.pi, 2*np.pi)    # 設定X軸的顯示範圍, from xmin to xmax
plt.ylim(-1.5, 1.5)          # 設定Y軸的顯示範圍, from ymin to ymax
#
xvalues = [0, 1*np.pi/6, 2*np.pi/6, 3*np.pi/6, 4*np.pi/6, 5*np.pi/6, np.pi, 
           7*np.pi/6, 8*np.pi/6, 9*np.pi/6, 10*np.pi/6, 11*np.pi/6, 2*np.pi]
#
yvalues = [-1.5,-1,-0.5,0,0.5,1,1.5]
#
plt.xticks(xvalues)
plt.yticks(yvalues)
plt.tick_params(labelsize=9)
#
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
#
plt.xlabel("RA Hours")
plt.ylabel("Sin(Dec)")
#
# get current axes, set X,Y same ratio & scale 
# 'equal' --> 0.5
plt.gca().set_aspect('equal', adjustable='box')
#
plt.grid(color='green', linewidth=0.25)
ax.grid(True)
# ax.grid(False)
ax.set_axis_on() # will turn on  grid line
# ax.set_axis_off()  # will turn off grid line
#
plt.savefig("09-1.png", format="png", dpi=150)
#
circle_1 = shapely_Arc((-np.pi/2,0), 1, 0, 360)
motif_line(circle_1, 'blue', 10, 0.8)
#
plt.savefig("09-2.png", format="png", dpi=150)
#
connecting_rod = LineString([(-np.pi/2,0), (-np.pi/2+1,0)])
motif_line(connecting_rod, 'red', 10, 0.8)
#
RA_line = LineString([(0,0), (2*np.pi,0)])
motif_line(RA_line, 'red', 10, 0.6)
#
plt.savefig("09-3.png", format="png", dpi=150)
#
ra_radian   = []
dec_degrees = []
df = pd.read_excel('./Ecliptic_ra-radian_dec-degree.xlsx', sheet_name='ecliptic')
#
for i in range(0, len(df)) :
    ra_radian.append(  df.at[i, 'ra_radian'])
    dec_degrees.append(df.at[i, 'dec_degrees'])
# end for
#
for i in range(1, len(df)) :
    ra_radian.append(  df.at[i, 'ra_radian'] + 2*np.pi)
    dec_degrees.append(df.at[i, 'dec_degrees'])
# end for
#
for i in range(1, len(df)) :
    ra_radian.append(  df.at[i, 'ra_radian'] + 4*np.pi)
    dec_degrees.append(df.at[i, 'dec_degrees'])
# end for
#
# function that draws each frame of the animation
i = 0 ; j = 0
degree = np.pi/180
def animate(i):
    ax.clear()
    #
    plt.xlim(-np.pi, 2*np.pi)    # 設定X軸的顯示範圍, from xmin to xmax
    plt.ylim(-1.5, 1.5)          # 設定Y軸的顯示範圍, from ymin to ymax
    #
    xvalues = [0, 1*np.pi/6, 2*np.pi/6, 3*np.pi/6, 4*np.pi/6, 5*np.pi/6, np.pi, 
               7*np.pi/6, 8*np.pi/6, 9*np.pi/6, 10*np.pi/6, 11*np.pi/6, 2*np.pi]
    #
    yvalues = [-1.5,-1,-0.5,0,0.5,1,1.5]
    plt.xticks(xvalues)
    plt.yticks(yvalues)
    #
    ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
    plt.tick_params(labelsize=9)
    #
    plt.xlabel("RA Hours")
    plt.ylabel("Sin(Dec)")
    #
    # get current axes, set X,Y same ratio & scale 
    plt.gca().set_aspect('equal', adjustable='box')
    #
    plt.grid(color='green', linewidth=0.25)
    ax.grid(True)
    # ax.grid(False)
    ax.set_axis_on() # will turn on  grid line
    # ax.set_axis_off()  # will turn off grid line
    #
    RA_line = LineString([(0,0), (2*np.pi,0)])
    motif_line(RA_line, 'red', 10, 0.6)
    #
    ax.set_title('Ecliptic Graph', fontsize=10, color='darkviolet')
    #
    motif_line(circle_1, 'blue', 10, 0.6)
    #
    rod_x  = np.cos(ra_radian[i]) - np.pi/2
    rod_y  = np.sin(dec_degrees[i]*degree)        
    #
    # -----------------------
    # 橢圓的指針, 從中心到圓周
    rod_xs = [-np.pi/2]
    rod_ys = [0]
    rod_xs.append(rod_x)
    rod_ys.append(rod_y)
    # -----------------------
    # 橢圓的圓周
    rod_XS.append(rod_x)
    rod_YS.append(rod_y)
    # -----------------------
    # 連桿的起迄值
    link_xs = [rod_x]
    link_ys = [rod_y]
    link_x  = 0
    link_y  = rod_y    
    link_xs.append(link_x)
    link_ys.append(link_y)    
    # -----------------------
    if i == 0 :
        j = 1
    else :
        sin_XS.append(ra_radian[i])
        sin_YS.insert(0,np.sin(dec_degrees[i]*degree))
    # end if
    #
    # 橢圓的指針
    ax.plot(rod_xs, rod_ys,  color='darkviolet', linewidth=0.6)
    # 橢圓的圓周
    ax.plot(rod_XS, rod_YS,  color='chocolate',  linewidth=1.0)
    # 連桿
    ax.plot(link_xs, link_ys, color='black', linewidth=0.6)
    # 黃道的軌跡
    ax.plot(sin_XS,  sin_YS,  color='blue',  linewidth=0.8)
# end def
#
sin_XS = [0]
sin_YS = [0]
rod_XS = []
rod_YS = []
#
# run the animation
i = 0
ani = animation.FuncAnimation(fig, animate, frames=366*2, interval=20, repeat=False)
#
print('Done')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Done
